<center><img src='https://i.postimg.cc/TPR1n1rp/AI-Tech-PL-RGB.png' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://i.postimg.cc/Gpq2KRQz/logotypy-aitech.jpg'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
</center>

# Lab 08: Imitation Learning

In this lab, we look into the problem of learning from expert demonstrations.

- Find a policy $\pi(a | s)$ that best imitates the expert policy $\pi^*(a | s)$ in the given environment.
- It's worth noting, that we don't need access to the environment rewards.

Major Imitation Learning techniques are:

1. Behavioural Cloning,
1. Imitation Learning via Interactive Demonstrator e.g. SMILe (Ross and Bagnell, 2010) or DAgger (Ross et al., 2011),
1. Inverse Reinforcement Learning -- out of scope of this lab.

We will solve the Ant problem, shown below, examining the first two approaches.

## Install dependencies

In [ ]:
!pip -q install gymnasium[mujoco]
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 56.7 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade torch==2.6.0
!pip install --upgrade numpy==1.26.4
!pip install --upgrade gymnasium==0.29.1

In [ ]:
!git clone --branch torch-2.6-fix https://github.com/lychanl/sample-factory.git

Cloning into 'sample-factory'...
remote: Enumerating objects: 17613, done.
remote: Counting objects: 100% (2165/2165), done.
remote: Compressing objects: 100% (328/328), done.
remote: Total 17613 (delta 1940), reused 1838 (delta 1837), pack-reused 15448 (from 2)
Receiving objects: 100% (17613/17613), 99.05 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (13898/13898), done.


In [ ]:
!pip install -q ./sample-factory[mujoco]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s

In [ ]:
%cd sample-factory

/content/sample-factory


## Download Expert

In [ ]:
!python -m sample_factory.huggingface.load_from_hub -r LLParallax/sf_Ant

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/LLParallax/sf_Ant into local empty directory.
[2025-04-24 12:35:14,255][04939] The repository LLParallax/sf_Ant has been cloned to ./train_dir/sf_Ant


In [ ]:
import functools

import torch

from sample_factory.algo.learning.learner import Learner
from sample_factory.algo.utils.env_info import extract_env_info
from sample_factory.algo.utils.make_env import make_env_func_batched
from sample_factory.algo.utils.rl_utils import prepare_and_normalize_obs
from sample_factory.cfg.arguments import load_from_checkpoint
from sample_factory.model.actor_critic import create_actor_critic
from sample_factory.model.model_utils import get_rnn_size
from sample_factory.utils.attr_dict import AttrDict
from sample_factory.utils.typing import Config


def create_expert(cfg):
    cfg = load_from_checkpoint(cfg)

    cfg.num_envs = 1

    env = make_env_func_batched(
        cfg, env_config=AttrDict(worker_index=0, vector_index=0, env_id=0), render_mode=None
    )

    if hasattr(env.unwrapped, "reset_on_init"):
        # reset call ruins the demo recording for VizDoom
        env.unwrapped.reset_on_init = False

    actor_critic = create_actor_critic(cfg, env.observation_space, env.action_space)
    actor_critic.eval()

    device = torch.device("cpu" if cfg.device == "cpu" else "cuda")
    actor_critic.model_to_device(device)

    policy_id = cfg.policy_index
    name_prefix = dict(latest="checkpoint", best="best")[cfg.load_checkpoint_kind]
    checkpoints = Learner.get_checkpoints(Learner.checkpoint_dir(cfg, policy_id), f"{name_prefix}_*")
    checkpoint_dict = Learner.load_checkpoint(checkpoints, device)
    actor_critic.load_state_dict(checkpoint_dict["model"])
    return actor_critic


def get_expert_actions(obs, cfg: Config, actor_critic, env, env_info, device):
    rnn_states = torch.zeros([env.num_agents, get_rnn_size(cfg)], dtype=torch.float32, device=device)

    obs = {"obs": obs}
    with torch.no_grad():
        normalized_obs = prepare_and_normalize_obs(actor_critic, obs)
        policy_outputs = actor_critic(normalized_obs, rnn_states)

        # sample actions from the distribution by default
        actions = policy_outputs["actions"]
    return actions

## Load expert model

In [ ]:
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sf_examples.mujoco.mujoco_params import add_mujoco_env_args, mujoco_override_defaults
from sf_examples.mujoco.train_mujoco import register_mujoco_components
from sf_examples.mujoco.mujoco_utils import MUJOCO_ENVS, make_mujoco_env


def register_mujoco_components():
    for env in MUJOCO_ENVS:
        register_env(env.name, make_mujoco_env)


register_mujoco_components()
argv = ["--algo=APPO", "--env=mujoco_ant", "--experiment=sf_Ant", "--train_dir=train_dir", "--no_render"]
parser, partial_cfg = parse_sf_args(argv=argv, evaluation=True)
add_mujoco_env_args(partial_cfg.env, parser)
mujoco_override_defaults(partial_cfg.env, parser)
cfg = parse_full_cfg(parser, argv=argv)
expert = create_expert(cfg)

[2025-04-24 12:48:16,271][08395] Loading existing experiment configuration from train_dir/sf_Ant/config.json
[2025-04-24 12:48:16,272][08395] Overriding arg 'experiment' with value 'sf_Ant' passed from command line
[2025-04-24 12:48:16,274][08395] Overriding arg 'train_dir' with value 'train_dir' passed from command line
[2025-04-24 12:48:16,275][08395] Adding new argument 'wandb_dir'='/content/sample-factory/wandb' that is not in the saved config file!
[2025-04-24 12:48:16,276][08395] Adding new argument 'fps'=0 that is not in the saved config file!
[2025-04-24 12:48:16,277][08395] Adding new argument 'eval_env_frameskip'=None that is not in the saved config file!
[2025-04-24 12:48:16,278][08395] Adding new argument 'no_render'=True that is not in the saved config file!
[2025-04-24 12:48:16,279][08395] Adding new argument 'save_video'=False that is not in the saved config file!
[2025-04-24 12:48:16,280][08395] Adding new argument 'video_frames'=1000000000.0 that is not in the saved co

## Helpers
collecting data  

evaluation

In [ ]:
import time

from IPython import display as ipydisplay

import torch
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import animation


@torch.no_grad()
def run_policy (env, model, total_steps=10000, verbose=True):
    obs_array = np.empty([total_steps, *env.observation_space.shape])
    act_array = np.empty([total_steps, env.action_space.shape[0]])
    rew_array = np.empty([total_steps, 1])
    done_array = np.empty([total_steps, 1])

    iter_time = time.time()
    done = True
    for i in range(total_steps):
        if verbose and (i + 1) % 1000 == 0:
            steps_per_second = 1000 / (time.time() - iter_time)
            print(f'Step {i + 1}/{total_steps}, Steps per second: {steps_per_second}')
            iter_time = time.time()

        if done:
            obs, info = env.reset()

        act = model(torch.from_numpy(obs).unsqueeze(0).float())[0].detach().cpu().numpy()
        obs_, rew, terminated, truncated, _ = env.step(act)
        done = terminated or truncated

        obs_array[i] = obs
        act_array[i] = act
        rew_array[i] = rew
        done_array[i] = float(done)

        obs = obs_

    return obs_array, act_array, rew_array, done_array

def calculate_returns(rew, done):
    rew_cumsum = np.cumsum(rew)[:, None]
    ret_cumsum = rew_cumsum * done
    ret_cumsum_trimed = ret_cumsum[np.nonzero(ret_cumsum)]
    ret_cumsum_trimed[1:] -= ret_cumsum_trimed[:-1]
    return ret_cumsum_trimed

def evaluate_agent(env, model, verbose=False):
    _, _, rew, done = run_policy(env, model, total_steps=50000, verbose=verbose)
    rets = calculate_returns(rew, done)

    print(f'Num. episodes: {len(rets)}')
    print(f'Avg. return: {np.mean(rets)}')
    print(f'Max. return: {np.max(rets)}')
    print(f'Min. return: {np.min(rets)}')

@torch.no_grad()
def collect_frames(eval_env, model, num_frames=2000):
    state, _ = eval_env.reset()
    state = torch.from_numpy(np.array(state)).float()
    frames = []

    for _ in range(num_frames):
        frames.append(eval_env.render())

        action = model(state.unsqueeze(0))[0]
        next_state, reward, terminal, truncate, info = eval_env.step(action.detach().cpu().numpy())

        if terminal or truncate:
            state, _ = eval_env.reset()
        state = next_state
        state = torch.from_numpy(np.array(state)).float()

    return frames

def display_frames_as_video(frames):
    """
    Displays a list of frames as a video.
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
    ipydisplay.display(ipydisplay.HTML(anim.to_jshtml()))

## 1. Behavior Clonning

Algorithm

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.

### Create model

In [ ]:
import torch
import torch.nn as nn


class MLP(nn.Module):
    def __init__(self, input_shape, output_size, hidden_sizes=(256, 256), hidden_activation=nn.Tanh(), output_activation=None, l2_weight=0.0001):
        super(MLP, self).__init__()
        self.layers = nn.Sequential()

        # Input layer
        self.layers.add_module("input", nn.Linear(input_shape, hidden_sizes[0]))
        self.layers.add_module("input_activation", hidden_activation)

        # Hidden layers
        layer_sizes = zip(hidden_sizes[:-1], hidden_sizes[1:])
        for i, (h1, h2) in enumerate(layer_sizes):
            self.layers.add_module(f"hidden_{i}", nn.Linear(h1, h2))
            self.layers.add_module(f"activation_{i}", hidden_activation)

        # Output layer
        self.layers.add_module("output", nn.Linear(hidden_sizes[-1], output_size))
        if output_activation is not None:
            self.layers.add_module("output_activation", output_activation)

        # Regularization
        self.l2_weight = l2_weight

    def forward(self, x):
        # Forward pass through the network
        x = self.layers(x)
        return x

    def l2_regularization(self):
        l2_reg = None
        for name, param in self.named_parameters():
            if 'weight' in name:
                if l2_reg is None:
                    l2_reg = param.norm(2)
                else:
                    l2_reg = l2_reg + param.norm(2)
        return self.l2_weight * l2_reg

### Function for training the model

In [ ]:
from torch.utils.data import DataLoader, TensorDataset


def train(obs, act, model, num_epochs=10, batch_size=32):
    obs_tensor = torch.tensor(obs, dtype=torch.float32)
    act_tensor = torch.tensor(act, dtype=torch.float32)

    dataset = TensorDataset(obs_tensor, act_tensor)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


    # Define the loss function and optimizer
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())

    # Training loop
    for epoch in range(num_epochs):
        for batch_idx, (x_batch, y_batch) in enumerate(data_loader):
            # Forward pass
            y_pred = model(x_batch)

            # Compute loss
            loss = loss_fn(y_pred, y_batch) + model.l2_regularization()

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print loss every epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
env = gym.make('Ant-v4')
env.num_agents = 1
env_info = extract_env_info(env, cfg)
device = torch.device("cpu" if cfg.device == "cpu" else "cuda")
collected_data = run_policy(env, functools.partial(get_expert_actions, cfg=cfg, actor_critic=expert, env=env, env_info=env_info, device=device), total_steps=10000)

Step 1000/10000, Steps per second: 379.65201811516704
Step 2000/10000, Steps per second: 426.8500788099053
Step 3000/10000, Steps per second: 528.6418946442903
Step 4000/10000, Steps per second: 554.1124077041671
Step 5000/10000, Steps per second: 535.8420951772597
Step 6000/10000, Steps per second: 531.941710313417
Step 7000/10000, Steps per second: 554.1015736982265
Step 8000/10000, Steps per second: 474.18550072066677
Step 9000/10000, Steps per second: 479.1545739551869
Step 10000/10000, Steps per second: 553.2128004216459


In [ ]:
obs, act, rewards, dones = collected_data

# EXERCISE: Create model
model = MLP(obs.shape[1],act.shape[1])

train(obs, act, model)

Epoch 1/10, Loss: 0.021796569228172302
Epoch 2/10, Loss: 0.016460420563817024
Epoch 3/10, Loss: 0.02250201813876629
Epoch 4/10, Loss: 0.01154329814016819
Epoch 5/10, Loss: 0.012282459065318108
Epoch 6/10, Loss: 0.010156218893826008
Epoch 7/10, Loss: 0.010844487696886063
Epoch 8/10, Loss: 0.014313793741166592
Epoch 9/10, Loss: 0.01035749725997448
Epoch 10/10, Loss: 0.010213336907327175


In [ ]:
evaluate_agent(env, model)

Num. episodes: 71
Avg. return: 3093.519522819137
Max. return: 5835.056780939398
Min. return: 83.65747508578352


### Exercise

Discuss the questions

1. In principle, do we need the expert policy for BC?

2. What are the problems with BC?

3. How can we help BC do better?


In [ ]:
# Collect the exploratory data
def exploratory(obs, **kwargs):
    """Adds the Gaussian noise to the expert actions."""
    actor_critic = kwargs['actor_critic']
    device = kwargs['device']
    rnn_states = torch.zeros([env.num_agents, get_rnn_size(cfg)], dtype=torch.float32, device=device)

    obs = {"obs": obs}
    with torch.no_grad():
        normalized_obs = prepare_and_normalize_obs(actor_critic, obs)
        policy_outputs = actor_critic(normalized_obs, rnn_states)

        # sample actions from the distribution by default
        action = policy_outputs["actions"]

    noise = np.random.normal(loc=0.0, scale=0.1, size=action.shape)
    actions = action.cpu().numpy().squeeze(0) + noise


    return action

expl_data = run_policy(env, functools.partial(exploratory, cfg=cfg, actor_critic=expert, env=env, env_info=env_info, device=device), total_steps=10000)

Step 1000/10000, Steps per second: 530.5566595720081
Step 2000/10000, Steps per second: 549.3742119854444
Step 3000/10000, Steps per second: 462.77798042615296
Step 4000/10000, Steps per second: 504.36368022197126
Step 5000/10000, Steps per second: 556.7721766833704
Step 6000/10000, Steps per second: 551.4555253902935
Step 7000/10000, Steps per second: 559.0335418472387
Step 8000/10000, Steps per second: 542.1329848629966
Step 9000/10000, Steps per second: 501.08788162471643
Step 10000/10000, Steps per second: 452.03963520437304


In [ ]:
obs_expl, act_expl, rewards, dones = expl_data
# Exercise: Run BC on the exploratory data

# ANSWER
model_expl = MLP(obs.shape[1],act.shape[1])

train(obs_expl, act_expl, model_expl)
# END ANSWER

Epoch 1/10, Loss: 0.02435445412993431
Epoch 2/10, Loss: 0.00960962101817131
Epoch 3/10, Loss: 0.012818087823688984
Epoch 4/10, Loss: 0.00984911061823368
Epoch 5/10, Loss: 0.013790028169751167
Epoch 6/10, Loss: 0.009992108680307865
Epoch 7/10, Loss: 0.010530606843531132
Epoch 8/10, Loss: 0.009741706773638725
Epoch 9/10, Loss: 0.009490123949944973
Epoch 10/10, Loss: 0.00864497385919094


In [ ]:
evaluate_agent(env, model_expl)

Num. episodes: 55
Avg. return: 4135.02754306536
Max. return: 5441.503706138639
Min. return: 248.34502075851196


### Exercise

Answer the questions

1. Why does it better?

2. How can we use the expert to further improve the data?


In [ ]:
# Exercise: Infere the expert actions on the exploratory observations
#           and run BC on it.

# ANSWER
model_expl2 = MLP(obs.shape[1],act.shape[1])

train(obs_expl, act, model_expl2)
# ANSWER END

Epoch 1/10, Loss: 0.10653416067361832
Epoch 2/10, Loss: 0.08438189327716827
Epoch 3/10, Loss: 0.07892758399248123
Epoch 4/10, Loss: 0.11598573625087738
Epoch 5/10, Loss: 0.124897800385952
Epoch 6/10, Loss: 0.11339180916547775
Epoch 7/10, Loss: 0.1143798977136612
Epoch 8/10, Loss: 0.13121433556079865
Epoch 9/10, Loss: 0.10024147480726242
Epoch 10/10, Loss: 0.09443077445030212


In [ ]:
evaluate_agent(env, model_expl2)

Num. episodes: 50
Avg. return: 973.5679341294891
Max. return: 977.7452602636986
Min. return: 969.3193223422713


### Exercise

Answer the questions

1. Did it help? Why?


1. How can you extend this idea?


## 2. Imitation Learning via Interactive Demostrator

[DAgger](https://www.ri.cmu.edu/pub_files/2011/4/Ross-AISTATS11-NoRegret.pdf)

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.
3. Collect the imitator data.
4. Infere the expert actions on the imitator data.
5. Fit the model to the extended dataset.
6. Repeat from 3.

In [ ]:
# We will pre-train on less expert data to keep the same dataset size
obs_ = obs[:2000,:]
act_ = act[:2000,:]

# EXERCISE: pretrain on first 2000 samples
# ANSWER
model_dagger = MLP(obs.shape[1],act.shape[1])

train(obs_, act_, model_dagger)
# END ANSWER

evaluate_agent(env, model_dagger)

Epoch 1/10, Loss: 0.015676820650696754
Epoch 2/10, Loss: 0.01980752684175968
Epoch 3/10, Loss: 0.020538102835416794
Epoch 4/10, Loss: 0.01718851923942566
Epoch 5/10, Loss: 0.01521988958120346
Epoch 6/10, Loss: 0.017059536650776863
Epoch 7/10, Loss: 0.0156804621219635
Epoch 8/10, Loss: 0.014178525656461716
Epoch 9/10, Loss: 0.016450518742203712
Epoch 10/10, Loss: 0.011872194707393646
Num. episodes: 52
Avg. return: 2135.42283528244
Max. return: 3514.7118929275457
Min. return: 354.0324307441406


In [ ]:
# Exercise: Implement DAgger

for i in range(4):
    print(f'\n### Iter. {i+1} ###')

    # ANSWER
    print('\n1. Data collection')
    obs_extra, _, _, _ = run_policy(env, model_dagger, total_steps=2000) # Collect 2k steps
    obs_ = np.concatenate((obs_, obs_extra), axis=0)
    act_expert = get_expert_actions(obs_extra, cfg=cfg, actor_critic=expert, env=env, env_info=env_info, device=device).cpu().numpy()
    act_ = np.concatenate((act_, act_expert), axis=0)
    print('\n2. Training')
    # reset model for fair comparison
    model_dagger = MLP(obs.shape[1],act.shape[1])

    train(obs_, act_, model_dagger)
    # END ANSWER

    print('\n3. Evaluation')
    evaluate_agent(env, model_dagger)


### Iter. 1 ###

1. Data collection
Step 1000/2000, Steps per second: 1472.4275108440604
Step 2000/2000, Steps per second: 1513.7896822174364

2. Training
Epoch 1/10, Loss: 0.020126618444919586
Epoch 2/10, Loss: 0.014334116131067276
Epoch 3/10, Loss: 0.016180135309696198
Epoch 4/10, Loss: 0.018862437456846237
Epoch 5/10, Loss: 0.013169465586543083
Epoch 6/10, Loss: 0.014183281920850277
Epoch 7/10, Loss: 0.014540482312440872
Epoch 8/10, Loss: 0.010575336404144764
Epoch 9/10, Loss: 0.017425434663891792
Epoch 10/10, Loss: 0.013572730123996735

3. Evaluation
Num. episodes: 66
Avg. return: 2325.631211924442
Max. return: 4792.506370554613
Min. return: 116.54724428863847

### Iter. 2 ###

1. Data collection
Step 1000/2000, Steps per second: 1510.4947285970236
Step 2000/2000, Steps per second: 1481.6548114612565

2. Training
Epoch 1/10, Loss: 0.02819034270942211
Epoch 2/10, Loss: 0.02891124226152897
Epoch 3/10, Loss: 0.022566312924027443
Epoch 4/10, Loss: 0.014326747506856918
Epoch 5/10, Loss

### Note

Training the expert with the PPO algorithm took 10M data samples (env. interactions). Here, we nearly match it with only 10k samples! Training from the expert can be much more efficient than reinforcement learning.